In [ ]:
import networkx

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pymedphys._experimental import tree

In [ ]:
module_dependencies = tree.get_module_dependencies()
internal_modules = set(module_dependencies.keys())

In [ ]:
# traversal of the pymedphys public API

root = 'pymedphys'
traversal_nodes = {root}
di_graph = networkx.DiGraph()
di_graph.add_node(root)

while traversal_nodes:
    node = traversal_nodes.pop()
    raw_dependencies = module_dependencies[node]
    
    for dependency in raw_dependencies:
        if (
            not dependency[2].startswith('_') and
            not dependency[1] in di_graph and
            dependency[1] in internal_modules
        ):
            traversal_nodes.add(dependency[1])
            di_graph.add_node(dependency[1])
            di_graph.add_edge(node, dependency[1])

In [ ]:
networkx.draw(di_graph)

In [ ]:
# module_dependencies

In [ ]:
def is_stable_public_api(module_name):
    not_stable_api_keys = ['._', 'beta', 'tests', 'docs', 'cli', 'experimental']
    for key in not_stable_api_keys:
        if key in module_name:
            return False

    return True

In [ ]:
public_modules = [
    key for key in module_dependencies.keys()
    if is_stable_public_api(key)
]

In [ ]:
# TODO:

# Get each publicly exposed API function/class.
# Draw a dependency tree for those publicly exposed APIs.
# Have a tree per API.

# The below doesn't achieve that yet.

In [ ]:
public_modules

In [ ]:
def append_module_dependencies_to_graph(di_graph, modules, dependency_map):
    new_modules_to_traverse = set()
    for module in modules:
        if module not in di_graph:
            di_graph.add_node(module)
        for dependency in dependency_map[module]:
            if dependency not in di_graph and dependency not in modules:
                di_graph.add_node(dependency)
                
                if dependency in dependency_map.keys():
                    new_modules_to_traverse.add(dependency)
                
            di_graph.add_edge(module, dependency)
            
    return di_graph, new_modules_to_traverse        

In [ ]:
di_graph = networkx.DiGraph()
modules_to_traverse = public_modules

while modules_to_traverse:
    di_graph, modules_to_traverse = append_module_dependencies_to_graph(
        di_graph, modules_to_traverse, module_dependencies)

In [ ]:
networkx.draw(di_graph)